# Offline Batch Recommender System

In this notebook, we will build a simple offline batch recsys that writes recommendations to Redis for quick retrieval later. The architecture diagram below shows how it comes together from a bird's eye view.

![](./img/OfflineBatchRecsys.png)

## Architecture
Recommender systems commonly have a *multi-stage pipeline*:
1) A fast **Candidate Retrieval Model** quickly truncates the large item catalog to a relevant set of hundreds (or thousands) of items.
2) Filtering is performed to remove undesirable or already-seen items.
3) A finely-tuned deep learning **Ranking Model** (i.e. more powerful) ranks the most likely items that are going to interacted with.
4) Results are ordered and returned to the user.

In this case, we will write recommendations to a key-value store (Redis) such that a client can request these in near "realtime" at a later point.

> WHY? This is especially useful for developers who can't afford the complexity of hosting a live multi-stage recsys OR need to get something up and running *quickly*

In this notebook, we will:

1) Prepare the [**Dataset**](#Dataset-Preparation)
2) Build a [**Candidate Retrieval Model**](#Candidate-Retrieval-Model)
3) Build a [**Ranking Model**](#Ranking-Model)
4) [**Write Recommendations**](#Write-Recommendations-to-Redis) to Redis (offline)
5) [**Fetch Recommendations**](#Fetch-Recommendations-from-Redis) from Redis
6) [**Export Models**](#Export-Models) for later use

*This notebook was created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container and was heavily based on the work done by the NVIDIA Merlin team [here](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb)*

## Dataset Preparation

We will use a synthetic dataset that mimicks the [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset. This allows us to tune it to our exact needs for demonstration/learning purposes.


### Importing Libraries

In [1]:
import os
import logging
import time

import nvtabular as nvt
import merlin.models.tf as mm
import tensorflow as tf

from nvtabular.ops import *

from merlin.datasets.synthetic import generate_data
from merlin.datasets.ecommerce import transform_aliccp
from merlin.models.utils.example_utils import workflow_fit_transform
from merlin.models.utils.dataset import unique_rows_by_features
from merlin.schema.tags import Tags
from merlin.io.dataset import Dataset


# disable INFO and DEBUG logging everywhere
logging.disable(logging.WARNING)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-13 17:36:22.281381: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2023-01-13 17:36:23.741804: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:991] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-13 17

### Generate Synthetic Ali-CCP Dataset

In [2]:
# Generate the synthetic data
NUM_ROWS = 1000000
TRAIN_SIZE = 0.7
VALID_SIZE = 0.3

train, valid = generate_data("aliccp-raw", NUM_ROWS, set_sizes=(TRAIN_SIZE, VALID_SIZE))

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [3]:
# Defin output path for data
DATA_DIR = os.environ['PWD'] +"/data/"
OUTPUT_DATA_DIR = os.path.join(DATA_DIR, "processed")
OUTPUT_RETRIEVAL_DATA_DIR = os.path.join(OUTPUT_DATA_DIR, "retrieval")
CATEGORY_TEMP_DIR = os.path.join(DATA_DIR, "categories")

In [4]:
# Define NVTabular Feature Transformation Pipeline

user_id_raw = ["user_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsUserFeatures()
item_id_raw = ["item_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsItemFeatures()

user_id = ["user_id"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsUserID()
item_id = ["item_id"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsItemID()

item_features = (
    ["item_category", "item_shop", "item_brand"] >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR) >> TagAsItemFeatures()
)

user_features = (
    [
        "user_shops",
        "user_profile",
        "user_group",
        "user_gender",
        "user_age",
        "user_consumption_2",
        "user_is_occupied",
        "user_geography",
        "user_intentions",
        "user_brands",
        "user_categories",
    ]
    >> Categorify(dtype="int32", out_path=CATEGORY_TEMP_DIR)
    >> TagAsUserFeatures()
)

targets = ["click"] >> AddMetadata(tags=[Tags.BINARY_CLASSIFICATION, "target"])

outputs = user_id + item_id + item_features + user_features +  user_id_raw + item_id_raw + targets

# add dropna op to filter rows with nulls
outputs = outputs >> Dropna()

With `transform_aliccp` function, we transform the raw dataset... applying the operators defined in the NVTabular workflow pipeline above. The processed parquet files are saved to the output path.

In [5]:
# Transform data and create files
transform_aliccp((train, valid), OUTPUT_DATA_DIR, nvt_workflow=outputs)

## Candidate Retrieval Model

We will use a **Two-Tower** model to infer a subset of relevant items from large item corpus for a given user. 

A Two-Tower Model consists of item (candidate) and user (query) encoder towers. With two towers, the model can learn representations (embeddings) for queries and candidates separately. 

<img src="https://d3i71xaburhd42.cloudfront.net/8c32706b6af49db5d9cd9217e5196f701e473537/2-Figure1-1.png"  width="30%">

Image from: [Mixed Negative Sampling for Learning Two-tower Neural Networks in Recommendations](https://www.semanticscholar.org/paper/Mixed-Negative-Sampling-for-Learning-Two-tower-in-Yang-Yi/29f080a1bb6df6f45afd82c443f72da745983bee)

### Feature Engineering for Candidate Retrieval

In [6]:
# Load Datasets from generated files
train_retrieval = Dataset(os.path.join(OUTPUT_DATA_DIR, "train", "*.parquet"))
valid_retrieval = Dataset(os.path.join(OUTPUT_DATA_DIR, "valid", "*.parquet"))

In [7]:
# Define NVTabular Feature Transformation Pipeline

inputs = train_retrieval.schema.column_names

# Select only positive interaction rows where click==1 in the dataset with Filter() operator
outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

# Execute the transformation workflow for both datasets
workflow = nvt.Workflow(outputs)
workflow.fit(train_retrieval)
workflow.transform(train_retrieval).to_parquet(
    output_path=os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "train")
)
workflow.transform(valid_retrieval).to_parquet(
    output_path=os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "valid")
)

*NVTabular exported the schema file, schema.pbtxt a protobuf text file, of our processed dataset. To learn more about the schema object and schema file you can explore [this notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb).*

In [8]:
# Read transformed parquet files as Dataset objects
train_retrieval = Dataset(os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "train", "*.parquet"), part_size="500MB")
valid_retrieval = Dataset(os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "valid", "*.parquet"), part_size="500MB")

Now we can use the `schema` object to define the model inputs. We select features with user and item tags, and exclude raw IDs and target column.

In [9]:
# Create model input schema
schema = train_retrieval.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER]).without(['user_id_raw', 'item_id_raw', 'click'])
train_retrieval.schema = schema
valid_retrieval.schema = schema

In [10]:
# Inspect the schema!
schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,user_id,"(Tags.USER, Tags.ID, Tags.USER_ID, Tags.CATEGO...",int32,False,False,None,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,752.0,65.0,0,751,user_id
1,item_id,"(Tags.ID, Tags.ITEM, Tags.ITEM_ID, Tags.CATEGO...",int32,False,False,None,0.0,0.0,0.0,/workdir/data/categories/categories/unique.ite...,759.0,66.0,0,758,item_id
2,item_category,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/categories/categories/unique.ite...,759.0,66.0,0,758,item_category
3,item_shop,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/categories/categories/unique.ite...,759.0,66.0,0,758,item_shop
4,item_brand,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/categories/categories/unique.ite...,759.0,66.0,0,758,item_brand
5,user_shops,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,752.0,65.0,0,751,user_shops
6,user_profile,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,49.0,16.0,0,48,user_profile
7,user_group,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,8.0,16.0,0,7,user_group
8,user_gender,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,3.0,16.0,0,2,user_gender
9,user_age,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,None,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,5.0,16.0,0,4,user_age


As expected, we shouldn't have any label/target data yet

In [11]:
label_names = schema.select_by_tag(Tags.TARGET).column_names
label_names

[]

### Designing Retrieval Model Architecture

The **Two-Tower** model consists of a **User tower** (where all user features are fed) and an **Item tower** (where all item features are fed).

The User tower generates an embedding for the User. Then it computes the positive interaction "score" (likelihood of interaction event) using the dot-product between the User embedding and the Item embedding, in addition to sampled "negative" Items within a batch.

##### About Negative Sampling

Many datasets for recommender systems contain implicit feedback with logs of user interactions like clicks, add-to-cart, purchases, music listening events, rather than explicit ratings that reflects user preferences over items. 

In Merlin Models -- NVIDIA provides some scalable negative sampling algorithms for this Item Retrieval task. In this example, we use the `in-batch` sampling algorithm which uses the items interacted by other users as negatives within the same mini-batch.

In [12]:
# Function to create the two tower retrieval model

def create_two_tower(tower_dim: int, encoder_dim: int, optimizer: str, k: int, tags) -> mm.TwoTowerModelV2:
    # User/Query Tower
    user_schema = schema.select_by_tag(tags.USER)
    # create user (query) tower input block
    user_inputs = mm.InputBlockV2(user_schema)
    # create user (query) encoder block
    query = mm.Encoder(
        user_inputs,
        mm.MLPBlock([encoder_dim, tower_dim], no_activation_last_layer=True)
    )

    # Item/Candidate Tower
    item_schema = schema.select_by_tag(tags.ITEM)
    # create item (candidate) tower input block
    item_inputs = mm.InputBlockV2(item_schema)
    # create item (candidate) encoder block
    candidate = mm.Encoder(
        item_inputs,
        mm.MLPBlock([encoder_dim, tower_dim], no_activation_last_layer=True)
    )
    
    # Build Model Class
    model = mm.TwoTowerModelV2(query, candidate)
    model.compile(
        optimizer=optimizer,
        run_eagerly=False,
        loss="categorical_crossentropy",
        metrics=[mm.RecallAt(k), mm.NDCGAt(k)]
    )
    return model

**Notes:**
- `no_activation_last_layer:` when set True, no activation is used for top hidden layer. Learn more [here](https://storage.googleapis.com/pub-tools-public-publication-data/pdf/b9f4e78a8830fe5afcf2f0452862fb3c0d6584ea.pdf).
- In the `TwoTowerModelV2` function we did not set `negative_samplers` arg. By default, it uses contrastive learning and `in-batch` negative sampling strategy.
- Two metrics are used to judge the quality of the recommendations: **Normalized Discounted Cumulative Gain (NDCG@K)** and **Recall@K**.
    - NDCG@K accounts for rank of the relevant item in the recommendation list and is a more fine-grained metric than HR, which only verifies whether the relevant item is among the top-k items.
    - Recall (Also known as HitRate@K) when there is only one relevant item in the recommendation list. Recall just verifies whether the relevant item is among the top-k items.
- When we set `validation_data=valid` in the `model.fit()`, we compute evaluation metrics on validation set using the negative sampling strategy used for training. 

In [13]:
# Initialize model
retrieval_model = create_two_tower(
    tower_dim=64,
    encoder_dim=128,
    optimizer="adam",
    k=10,
    tags=Tags
)

# Fit model
retrieval_model.fit(train_retrieval, validation_data=valid_retrieval, batch_size=4096, epochs=2)

Epoch 1/2
86/86 [==============================] - 11s 58ms/step - loss: 8.2958 - recall_at_10: 0.0271 - ndcg_at_10: 0.0258 - regularization_loss: 0.0000e+00 - loss_batch: 8.2868 - val_loss: 8.2903 - val_recall_at_10: 0.0295 - val_ndcg_at_10: 0.0295 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.6540
Epoch 2/2
86/86 [==============================] - 4s 47ms/step - loss: 8.2953 - recall_at_10: 0.0306 - ndcg_at_10: 0.0305 - regularization_loss: 0.0000e+00 - loss_batch: 8.2863 - val_loss: 8.2908 - val_recall_at_10: 0.0206 - val_ndcg_at_10: 0.0194 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 7.6539


### Evaluate the model accuracy

The validation metric values during training are calculated given the positive and negative scores in each batch, and then averaged over batches per epoch. **That means validation metrics are not computed using the entire item catalog.**

To determine the exact accuracy, we need to compute the similarity score between a given query and all possible candidates. Below, by using the `topk_model` we can evaluate the trained retrieval model using the entire item catalog (brute force).

In [14]:
# Create candidate/item features for evaluation
candidate_features = unique_rows_by_features(train_retrieval, Tags.ITEM, Tags.ITEM_ID)

# Here's a sneek peek of the item features data
candidate_features.head()

,item_id,item_category,item_shop,item_brand
24,1,1,1,1
93,2,2,2,2
11,3,3,3,3
3,4,4,4,4
37,5,5,5,5


In [15]:
# Convert model to a top_k_encoder
topk_model = retrieval_model.to_top_k_encoder(candidate_features, k=20, batch_size=128)
topk_model.compile(run_eagerly=False)

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [16]:
# Create data loader for validation data
eval_loader = mm.Loader(valid_retrieval, batch_size=1024).map(mm.ToTarget(schema, "item_id"))

# Evaluation
metrics = topk_model.evaluate(eval_loader, return_dict=True)
metrics

147/147 [==============================] - 3s 13ms/step - loss: 0.4620 - recall_at_10: 0.1104 - mrr_at_10: 0.0386 - ndcg_at_10: 0.0552 - map_at_10: 0.0386 - precision_at_10: 0.0110 - regularization_loss: 0.0000e+00 - loss_batch: 0.4596


{'loss': 0.46201813220977783,
 'recall_at_10': 0.11041296273469925,
 'mrr_at_10': 0.038629066199064255,
 'ndcg_at_10': 0.055212415754795074,
 'map_at_10': 0.038629066199064255,
 'precision_at_10': 0.01104129757732153,
 'regularization_loss': 0.0,
 'loss_batch': 0.27507323026657104}

### Generate top-K recommendations

Let's generate top-K (k=20 in our example) recommendations for a given batch of 8 samples.
- The `to_top_k_encoder()` method uses the item/candidate features dataset to compute and store all item/candidate embeddings in an index.
- The forward method of `topk_model` takes as the query/user features as input, and computes the dot product scores between the given query/user embeddings and all the candidates of the top-k index.
- Then, it returns the top-k (k=20) item ids with the highest scores.

In [17]:
# Create query/user features for evaluation
user_features = unique_rows_by_features(valid_retrieval, Tags.USER, Tags.USER_ID)
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories
4400,0,0,20,3,1,2,1,1,1,0,0,0
9,1,1,1,1,1,1,1,1,1,1,1,1
26,2,2,1,1,1,1,1,1,1,2,2,2
53,3,3,1,1,1,1,1,1,1,3,3,3
25,4,4,1,1,1,1,1,1,1,4,4,4


In [18]:
# Check out one batch of 8 Users
loader = mm.Loader(user_features, batch_size=8, shuffle=False)
batch = next(iter(loader))
print(batch[0]['user_id'])

tf.Tensor(
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]], shape=(8, 1), dtype=int32)


The recommended top 20 item ids and scores are returned below from the candidate retrieval model for each of the 8 selected users (from the validation set).

In [19]:
scores, reccommended_item_ids = topk_model(batch[0])

In [20]:
# Recommended Items for the batch of 8 Users
reccommended_item_ids

<tf.Tensor: shape=(8, 20), dtype=int32, numpy=
array([[  9,   8, 133,   1,  31,  12, 399,  28, 407,  41,   3, 377, 169,
        437,  10, 265, 328,  13, 220, 235],
       [ 46, 150,   6,  49, 248, 121,   5,  41,  40, 736,  17, 149, 357,
         93, 183,  13, 434, 293, 103, 199],
       [ 49, 150,  46, 736,   6, 357,  17,   5, 248,   7, 121,  93, 119,
         13, 670, 183, 622, 149,  40, 224],
       [ 11,  49,   4,   6,  14,  38,  40,  19,  41, 138, 154, 271, 149,
        377, 432, 736, 475, 352,  31, 353],
       [ 53, 227,   2,  10,  51, 252, 188,  98, 114,  11,  79, 164, 260,
        372, 628, 217, 141, 400, 465,  29],
       [ 41,  28,   4,  12, 158,  90, 199, 297, 286,   1, 335, 378,  49,
        419, 674, 328, 358,   3, 212, 679],
       [ 41, 199,   9,  28, 437,  31,   8, 204,  13,  84,   1, 172,   5,
        475,  39, 297, 235, 476, 250, 185],
       [ 20,  46, 119, 541,  49, 150, 231,  13, 283,  41, 367, 144, 351,
         36, 161,  86,  16,  28, 206, 405]], dtype=int32)>

In [21]:
# Recommendation "scores" (higher is better) for each recommended item for the batch of 8 Users
scores

<tf.Tensor: shape=(8, 20), dtype=float32, numpy=
array([[0.04822209, 0.03811773, 0.03137591, 0.02973269, 0.0297016 ,
        0.0294863 , 0.02825383, 0.02756989, 0.02693252, 0.02579422,
        0.02547731, 0.02464605, 0.02369959, 0.02304306, 0.02284535,
        0.02267022, 0.02261037, 0.0224703 , 0.02204175, 0.02201148],
       [0.07010886, 0.06604874, 0.0622276 , 0.06199696, 0.05799136,
        0.05622398, 0.05363223, 0.05351985, 0.05214779, 0.0504364 ,
        0.04894517, 0.0485014 , 0.04741625, 0.04698368, 0.04684058,
        0.04681647, 0.04489934, 0.04478962, 0.04435818, 0.04322811],
       [0.05997518, 0.05802372, 0.05706425, 0.04934998, 0.04920235,
        0.04612496, 0.04592583, 0.0453655 , 0.04535462, 0.04380718,
        0.04362945, 0.04173674, 0.04139153, 0.04094845, 0.04082045,
        0.03983249, 0.03905755, 0.03840206, 0.03795295, 0.03658341],
       [0.05653799, 0.0493288 , 0.03923938, 0.0361588 , 0.03259742,
        0.03182421, 0.03178427, 0.02825179, 0.0281584 , 0.027129

At this point - we now have a trained **Candidate Retrieval Model**. Input a **User** and find the topK most likely **Items** to be interacted with. These will serve as inputs to the next model in the pipeline...

## Ranking Model

We will use a Deep Learning Recommendation Model [(DLRM)](https://arxiv.org/abs/1906.00091) architecture (published by Facebook/Meta in 2019) to score and rank User/Item pairs. The model was introduced as a personalization deep learning model that uses embeddings to process sparse features that represent categorical data and a multilayer perceptron (MLP) to process dense features, then interacts these features explicitly using the statistical techniques proposed in [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5694074). To learn more about DLRM architetcture please visit [this notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/04-Exporting-ranking-models.ipynb) in the Merlin Models GH repo.


### Feature Engineering for DLRM

In [22]:
# Define train and valid dataset objects
train_rank = Dataset(os.path.join(OUTPUT_DATA_DIR, "train", "*.parquet"), part_size="500MB")
valid_rank = Dataset(os.path.join(OUTPUT_DATA_DIR, "valid", "*.parquet"), part_size="500MB")

# Define schema object
schema = train_rank.schema.without(['user_id_raw', 'item_id_raw'])

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


In [23]:
# Inspect schema - DLRM takes all of these features as input!
schema

,name,tags,dtype,is_list,is_ragged,properties.num_buckets,properties.freq_threshold,properties.max_size,properties.start_index,properties.cat_path,properties.embedding_sizes.cardinality,properties.embedding_sizes.dimension,properties.domain.min,properties.domain.max,properties.domain.name
0,user_id,"(Tags.USER, Tags.ID, Tags.USER_ID, Tags.CATEGO...",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,752.0,65.0,0,751,user_id
1,item_id,"(Tags.ID, Tags.ITEM, Tags.ITEM_ID, Tags.CATEGO...",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/categories/categories/unique.ite...,759.0,66.0,0,758,item_id
2,item_category,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/categories/categories/unique.ite...,759.0,66.0,0,758,item_category
3,item_shop,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/categories/categories/unique.ite...,759.0,66.0,0,758,item_shop
4,item_brand,"(Tags.ITEM, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/categories/categories/unique.ite...,759.0,66.0,0,758,item_brand
5,user_shops,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,752.0,65.0,0,751,user_shops
6,user_profile,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,49.0,16.0,0,48,user_profile
7,user_group,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,8.0,16.0,0,7,user_group
8,user_gender,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,3.0,16.0,0,2,user_gender
9,user_age,"(Tags.USER, Tags.CATEGORICAL)",int32,False,False,NaN,0.0,0.0,0.0,/workdir/data/categories/categories/unique.use...,5.0,16.0,0,4,user_age


In [24]:
# Target column here is "click" (binary classification)
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'click'

### Building DLRM

In [25]:
def create_dlrm(optimizer: str, schema, target_column: str, embedding_dim: int):
    model = mm.DLRMModel(
        schema,
        embedding_dim=embedding_dim,
        bottom_block=mm.MLPBlock([128, 64]),
        top_block=mm.MLPBlock([128, 64, 32]),
        prediction_tasks=mm.BinaryClassificationTask(target_column),
    )
    model.compile(optimizer=optimizer, run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
    return model


In [26]:
dlrm = create_dlrm(
    optimizer="adam",
    schema=schema,
    target_column=target_column,
    embedding_dim=64
)

dlrm.fit(train_rank, validation_data=valid_rank, batch_size=16 * 1024, epochs=3)

Epoch 1/3
43/43 [==============================] - 5s 40ms/step - loss: 0.6932 - auc: 0.4999 - regularization_loss: 0.0000e+00 - loss_batch: 0.6932 - val_loss: 0.6931 - val_auc: 0.5000 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.6932
Epoch 2/3
43/43 [==============================] - 1s 23ms/step - loss: 0.6931 - auc: 0.5031 - regularization_loss: 0.0000e+00 - loss_batch: 0.6931 - val_loss: 0.6932 - val_auc: 0.5008 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.6932
Epoch 3/3
43/43 [==============================] - 1s 22ms/step - loss: 0.6928 - auc: 0.5144 - regularization_loss: 0.0000e+00 - loss_batch: 0.6928 - val_loss: 0.6936 - val_auc: 0.5004 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 0.6936


In [27]:
# Check out one batch of users
loader = mm.Loader(valid_rank, batch_size=8, shuffle=False)
batch = next(iter(loader))
print(batch[0]['user_id'])

tf.Tensor(
[[ 7]
 [33]
 [51]
 [19]
 [25]
 [13]
 [38]
 [51]], shape=(8, 1), dtype=int32)


In [28]:
# Test the DLRM model I/O with the user batch
dlrm(batch[0])

<tf.Tensor: shape=(8, 1), dtype=float32, numpy=
array([[0.5076402 ],
       [0.4976217 ],
       [0.51428837],
       [0.50732696],
       [0.5092731 ],
       [0.51470715],
       [0.5204432 ],
       [0.5047823 ]], dtype=float32)>

### Test the entire pipeline

So now that we have a **Candidate Retrieval Model** and a **Ranking Model** (DLRM) we can test the end to end flow locally. In order to do this live, like in the next multi-stage section, you need a **Feature Store** (Redis/Feast), **ANN Index** (Redis), and means to host/serve the entire DAG of operations (Triton).

*Below is a very brute force approach for a single batch of users.*

In [29]:
# Test full pipeline
import numpy as np

# Reload Dataset
train_retrieval = Dataset(os.path.join(OUTPUT_RETRIEVAL_DATA_DIR, "train", "*.parquet"), part_size="500MB")
schema = train_retrieval.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER]).without(['click'])
train_retrieval.schema = schema

# User and Item Feature "Stores"
item_fs = unique_rows_by_features(train_retrieval, Tags.ITEM, Tags.ITEM_ID).to_ddf().compute()
user_fs = unique_rows_by_features(train_retrieval, Tags.USER, Tags.USER_ID).to_ddf().compute()

In [30]:
# User batch loader
user_loader = mm.Loader(unique_rows_by_features(train_retrieval, Tags.USER, Tags.USER_ID), batch_size=8, shuffle=False)

# Sample User batch
user_batch = next(iter(user_loader))
users = user_batch[0]['user_id']
users

<tf.Tensor: shape=(8, 1), dtype=int32, numpy=
array([[1],
       [2],
       [3],
       [4],
       [5],
       [6],
       [7],
       [8]], dtype=int32)>

In [31]:
# Retrieve candidate Items for each User
_, candidate_item_ids = topk_model(user_batch[0])
candidate_item_ids

<tf.Tensor: shape=(8, 20), dtype=int32, numpy=
array([[ 46, 150,   6,  49, 248, 121,   5,  41,  40, 736,  17, 149, 357,
         93, 183,  13, 434, 293, 103, 199],
       [ 49, 150,  46, 736,   6, 357,  17,   5, 248,   7, 121,  93, 119,
         13, 670, 183, 622, 149,  40, 224],
       [ 11,  49,   4,   6,  14,  38,  40,  19,  41, 138, 154, 271, 149,
        377, 432, 736, 475, 352,  31, 353],
       [ 53, 227,   2,  10,  51, 252, 188,  98, 114,  11,  79, 164, 260,
        372, 628, 217, 141, 400, 465,  29],
       [ 41,  28,   4,  12, 158,  90, 199, 297, 286,   1, 335, 378,  49,
        419, 674, 328, 358,   3, 212, 679],
       [ 41, 199,   9,  28, 437,  31,   8, 204,  13,  84,   1, 172,   5,
        475,  39, 297, 235, 476, 250, 185],
       [ 20,  46, 119, 541,  49, 150, 231,  13, 283,  41, 367, 144, 351,
         36, 161,  86,  16,  28, 206, 405],
       [ 46,  11,  49,  13,   6, 104, 223, 357,  14, 148, 106, 121, 736,
        701, 391,  91,  26, 186, 388, 449]], dtype=int32)>

In [32]:
# For each user + candidate items, score with the DLRM
for user, candidates in zip(users.numpy(), candidate_item_ids.numpy()):
    
    num_recs = len(candidates)
    user_id = user[0]
    
    # Pull user features from feature store
    user_features = user_fs[user_fs.user_id == user_id]
    raw_user_id = user_features.user_id_raw.to_numpy()[0]
    user_features = user_features.append([user_features]*(num_recs-1), ignore_index=True)

    # Pull item features from feature store
    item_features = item_fs[item_fs.item_id.isin(candidates)].reset_index(drop=True)
    raw_item_ids = item_features.item_id_raw.to_numpy()
    
    # combined features
    item_features[user_features.columns] = user_features
    item_features = Dataset(item_features)
    item_features.schema = schema.without(['click'])
    
    # Score with DLRM - TODO -- can we do this without the Loader???
    inputs = mm.Loader(item_features, batch_size=num_recs)
    inputs = next(iter(inputs))
    scores = dlrm(inputs[0]).numpy().reshape(-1)
    
    # Rank
    recs = raw_item_ids[np.where(scores > 0.5)].tolist() 

/usr/local/lib/python3.8/dist-packages/cudf/core/indexed_frame.py:2328: FutureWarning: The append method is deprecated and will be removed in a future version. Use cudf.concat instead.
  warnings.warn(


In [33]:
# Look at one of the recommendations for a User
recs

[10, 14, 3, 16, 27, 46, 49, 91, 104, 121, 149, 180, 228, 388, 398, 508, 1006]

## Write Recommendations to Redis

Redis is used (low latency k-v store) to persist recommendations for each User.


In [34]:
import asyncio
import redis.asyncio as redis
from redis.commands.json.path import Path

def generate_recs(topk_model, dlrm, input_data, batch_size: int, tags):
    user_loader = mm.Loader(unique_rows_by_features(input_data, tags.USER, tags.USER_ID), batch_size=batch_size, shuffle=False)
    # Load a batch
    for batch in user_loader:
        
        users = batch[0]['user_id']
        
        # Generate candidates per user
        _, candidate_item_ids = topk_model(batch[0])
        
        # For each user + candidate items, score with the DLRM
        for user, candidates in zip(users.numpy(), candidate_item_ids.numpy()):
            try:
                num_recs = len(candidates)
                user_id = user[0]

                # Pull user features from feature store
                user_features = user_fs[user_fs.user_id == user_id]
                raw_user_id = user_features.user_id_raw.to_numpy()[0]
                user_features = user_features.append([user_features]*(num_recs-1), ignore_index=True)

                # Pull item features from feature store
                item_features = item_fs[item_fs.item_id.isin(candidates)].reset_index(drop=True)
                raw_item_ids = item_features.item_id_raw.to_numpy()

                # combined features
                item_features[user_features.columns] = user_features
                item_features = Dataset(item_features)
                item_features.schema = schema.without(['click'])

                # Score with DLRM - TODO -- can we do this without the Loader???
                inputs = mm.Loader(item_features, batch_size=num_recs)
                inputs = next(iter(inputs))
                scores = dlrm(inputs[0]).numpy().reshape(-1)

                # Rank
                recs = raw_item_ids[np.where(scores > 0.5)].tolist() 

                yield raw_user_id, recs
            except Exception as e:
                logging.info(user_id, str(e))

In [35]:
# Test Recommendation Generator
next(generate_recs(topk_model, dlrm, train_retrieval, 32, Tags))

# SEE BELOW: User ID --> Recommended IDs

(7, [9, 10, 3, 19, 40, 41, 49, 93, 150, 183, 196, 236, 309, 339, 1006])

In [36]:
async def store_recommendations(rec_gen, redis_conn: redis.Redis):
    """
    Store recommendations generated for each User.
    """
    async def store(user_id: str, recs: list):
        """
        Store and individual User's latest recommendations in Redis.
        """
        entry = {
            "user_id": int(user_id),
            "recommendations": [int(rec) for rec in recs]
        }
        # Set the JSON object in Redis
        await redis_conn.json().set(f"USER:{user_id}", Path.root_path(), entry)
    # Write to Redis
    for user_id, recs in top_recs_per_user:
        await store(user_id, recs)

In [37]:
redis_conn = redis.Redis(
    host="redis-inference-store",
    port=6379,
    decode_responses=True
)

# Create Recommendation Denerator
top_recs_per_user = generate_recs(topk_model, dlrm, train_retrieval, 32, Tags)

# Run the process - may take a few minutes
await store_recommendations(top_recs_per_user, redis_conn=redis_conn)

## Fetch Recommendations from Redis

Now that you've written a list of recommended item IDs for each User - we can use the Redis CLI to take a peek at one example. Your application can now fetch these when needed (at low latency).

In [48]:
!redis-cli -h redis-inference-store -p 6379 JSON.GET USER:1

"{\"user_id\":1,\"recommendations\":[21,1,35,44,51,57,186,228,264,307,308,342,422,481,693]}"
"{\"user_id\":1,\"recommendations\":[21,1,35,44,51,57,186,228,264,307,308,342,422,481,693]}"
"{\"user_id\":1,\"recommendations\":[21,1,35,44,51,57,186,228,264,307,308,342,422,481,693]}"
"{\"user_id\":1,\"recommendations\":[21,1,35,44,51,57,186,228,264,307,308,342,422,481,693]}"
"{\"user_id\":1,\"recommendations\":[21,1,35,44,51,57,186,228,264,307,308,342,422,481,693]}"
"{\"user_id\":1,\"recommendations\":[21,1,35,44,51,57,186,228,264,307,308,342,422,481,693]}"
"{\"user_id\":1,\"recommendations\":[21,1,35,44,51,57,186,228,264,307,308,342,422,481,693]}"
"{\"user_id\":1,\"recommendations\":[21,1,35,44,51,57,186,228,264,307,308,342,422,481,693]}"
695 ms ± 2.66 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [56]:
# Simple benchmark with async Python client
times = []
for i in range(300):
    t = time.time()
    await redis_conn.json().get(f"USER:{i}")
    times.append(time.time()-t)

In [57]:
# Average read time from Redis --> less than a ms likely
np.average(times)

0.00019584973653157553

## Export Models

So far we have trained and evaluated our models for the recsys pipeline. We need to export the models and artifacts in order to deploy them "online" in the next notebook.

### Export Retrieval Model

We are able to save the user tower model as a TF model to disk. The user tower model is needed to generate a user embedding vector when a user feature vector <i>x</i> is fed into that model.

In [39]:
query_tower = retrieval_model.query_encoder
query_tower.save(os.path.join(DATA_DIR, "query_tower"))

## we can load back the saved model via the following script.
#query_tower_loaded = tf.keras.models.load_model(os.path.join(DATA_DIR, 'query_tower'))

### Export User features

With `unique_rows_by_features` utility function we can easily extract both unique user and item features tables as cuDF dataframes. Note that for user features table, we use `USER` and `USER_ID` tags. These will get stored in our Feature Store later on.

In [40]:
user_features = (
    unique_rows_by_features(train_retrieval, Tags.USER, Tags.USER_ID).compute().reset_index(drop=True)
)
user_features.head()

,user_id,user_shops,user_profile,user_group,user_gender,user_age,user_consumption_2,user_is_occupied,user_geography,user_intentions,user_brands,user_categories,user_id_raw
0,1,1,1,1,1,1,1,1,1,1,1,1,7
1,2,2,1,1,1,1,1,1,1,2,2,2,6
2,3,3,1,1,1,1,1,1,1,3,3,3,8
3,4,4,1,1,1,1,1,1,1,4,4,4,9
4,5,5,1,1,1,1,1,1,1,5,5,5,5


In [41]:
# save to disk
user_features.to_parquet(os.path.join(DATA_DIR, "user_features.parquet"))

### Export User Embeddings

In [42]:
queries = retrieval_model.query_embeddings(Dataset(user_features, schema=schema), batch_size=1024, index=Tags.USER_ID)
query_embs_df = queries.compute(scheduler="synchronous").reset_index()

/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.USER_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.USER: 'user'>, <Tags.ID: 'id'>].
  warnings.warn(


In [43]:
query_embs_df.head()

,user_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,-0.111068,-0.036823,-0.044900,0.035209,0.003688,-0.007081,-0.038696,-0.106746,-0.042413,...,-0.092540,-0.092408,-0.020449,0.094238,-0.075105,-0.003479,0.032159,0.001669,-0.096887,0.124208
1,2,-0.048456,-0.041089,-0.076468,0.049387,-0.020447,-0.054963,-0.032225,-0.085540,-0.034649,...,-0.070766,-0.094994,-0.016747,0.102674,-0.028168,0.004202,0.016498,-0.037883,-0.096017,0.093783
2,3,-0.009904,-0.088978,-0.037149,0.045837,-0.059934,-0.028095,0.003774,-0.123409,-0.045720,...,-0.059774,-0.095987,0.009015,0.034939,-0.029024,0.023600,-0.015245,0.045124,-0.120927,0.083832
3,4,-0.027886,-0.068725,0.009604,-0.000948,0.000705,-0.048250,0.041702,-0.029777,-0.026400,...,0.016240,0.037292,0.059883,0.037400,-0.179152,0.032167,-0.006257,0.030779,-0.139906,0.070979
4,5,-0.029951,-0.027248,-0.047780,0.076823,-0.119614,0.030432,0.011596,-0.190625,-0.072068,...,-0.024637,-0.130085,-0.041280,-0.013120,-0.052877,0.062012,-0.105109,0.161616,-0.123151,0.089686


### Export Item Features

In [44]:
item_features = (
    unique_rows_by_features(train_retrieval, Tags.ITEM, Tags.ITEM_ID).compute().reset_index(drop=True)
)
item_features.head()

,item_id,item_category,item_shop,item_brand,item_id_raw
0,1,1,1,1,7
1,2,2,2,2,8
2,3,3,3,3,6
3,4,4,4,4,5
4,5,5,5,5,9


In [45]:
# save to disk
item_features.to_parquet(os.path.join(DATA_DIR, "item_features.parquet"))

### Export Item Embeddings
These will get stored in our ANN Index (in Redis).

In [58]:
item_embs = retrieval_model.candidate_embeddings(Dataset(item_features, schema=schema), batch_size=1024, index=Tags.ITEM_ID)
item_embs_df = item_embs.compute(scheduler="synchronous")
item_embs_df.head()

/usr/local/lib/python3.8/dist-packages/google/protobuf/internal/python_message.py:513: RuntimeWarning: coroutine 'Redis.execute_command' was never awaited
  self._listener = message_listener_mod.NullMessageListener()
/usr/local/lib/python3.8/dist-packages/merlin/schema/tags.py:148: UserWarning: Compound tags like Tags.ITEM_ID have been deprecated and will be removed in a future version. Please use the atomic versions of these tags, like [<Tags.ITEM: 'item'>, <Tags.ID: 'id'>].
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
item_id,,,,,,,,,,,,,,,,,,,,,
1,0.053040,-0.072800,0.029477,0.061942,-0.083255,-0.027927,0.040919,0.049997,-0.029252,0.061744,...,0.049286,-0.087432,0.009700,-0.071424,0.024802,-0.012824,-0.045329,0.098826,0.028681,-0.000962
2,-0.022205,-0.073701,0.084334,-0.087279,0.050656,-0.088896,0.024600,0.115385,-0.007804,0.068305,...,0.069289,0.089851,0.059622,0.002280,-0.102191,0.056542,0.067240,-0.019867,-0.049110,-0.099910
3,0.009324,0.004293,-0.021806,0.011273,0.020458,0.000370,0.035672,0.035399,-0.019392,0.078260,...,-0.066249,-0.038092,0.028947,-0.004527,-0.007591,0.090114,-0.041121,0.030358,-0.016289,0.000246
4,0.028158,-0.015050,-0.010841,0.057088,-0.019317,0.015477,0.011777,-0.020076,-0.026838,0.057488,...,-0.004810,-0.044477,-0.003194,0.012644,-0.043281,0.032553,-0.011370,0.024755,-0.060624,-0.010108
5,-0.041253,0.041424,0.004513,-0.033554,0.051555,-0.000238,0.004218,0.086332,-0.030636,0.052590,...,-0.083374,0.008296,-0.003563,0.006302,0.002813,0.013661,0.040882,0.018407,-0.014042,-0.012307


In [59]:
# save to disk
item_embs_df.to_parquet(os.path.join(DATA_DIR, "item_embeddings.parquet"))

### Export DLRM

In [60]:
dlrm.save(os.path.join(DATA_DIR, "dlrm"))

## Conclusion

No we have learned how to train and evaluate your Two-Tower **Candidate Retrieval Model** and **DLRM**. We also wrote recommendations "offline" to a key-value store (Redis) for use in our applications.

![img](./img/OfflineBatchRecsys.png)

*This completes the **"Offline Batch Recommender System"** intro with Redis + NVIDIA*. In the next tutorial, we will focus on deploying these assets to the [Triton Inference Server](https://github.com/triton-inference-server/server) for live "online" recommendations.